# Исследование надёжности заёмщиков — анализ банковских данных.

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверить зависимость факта возврата кредита в срок от следующих факторов:
1. количество детей;
2. семейное положение;
3. уровнь дохода;
4. цель кредита;

**Ход исследования**

Данные с информацией о клиенте и взятом кредите хранятся в файле `data.csv`. 
Известно, что в данных есть пропущенные значения, могут встречаться артефакты (аномалии) и дубликаты.
Поэтому перед выявлением интересующих зависимостей понадобится обзор и предобработка данных. 

Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка наличия зависимостей.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import numpy as np
#credit_data = pd.read_csv('/datasets/data.csv')

try:
    credit_data = pd.read_csv('data_files/data.csv')  # Локальный путь
except:
    credit_data = pd.read_csv('/datasets/data.csv')  # Серверный путь


In [2]:
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов с различными типами данных.

Согласно документации к данным:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Из обзора данных сразу видны следующие недочеты: 
* Пропуски в столбцах `days_employed` и `total_income`
* Отрицательные значения и аномальные значения в столбце `days_employed`
* Использование разного регистра в столбце `education`


**Выводы**

В каждой строке таблицы — данные о клиенте банка. Большая часть колонок описывает клиетна: возраст, пол, инфоримцию о семье, образовании и работе. Также есть информация о цели получения кредита и наличии задолженности по возврату кредита. 

Предварительно можно утверждать, что, данных достаточно для установления зависимостей. Но в данных встречаются пропуски, аномальные значения и разный регистр при написании категоральных значений.

Следующие пункты будут направлены на устрание проблем в данных.

### Шаг 2.1 Заполнение пропусков

In [4]:
# Сразу приведем столбец 'education' к нижнему регистру
credit_data['education'] = credit_data['education'].str.lower()

In [5]:
credit_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в столбцах `days_employed` и `total_income`.

Возможные причины возникновения пропусков:
* ошибки при копировании или модификации файла
* опечатки или неправильный ввод данных

In [6]:
print('Доля пропущенных значений: {:.1%}'.format(credit_data['days_employed'].isna().mean() ) )

Доля пропущенных значений: 10.1%


Стоит отметить, что количесво пропусков в столбцах `total_income` и `days_employed` совпадают. При проверке, оказалось, что пропускив этих столбцах находятся в одних и тех же строках

In [7]:
# Подсчет пропусков в столбце total_income при условии, что в столце days_employed пропуск
credit_data[credit_data['days_employed'].isna()]['total_income'].isna().sum()

2174

Можно предположить, что люди намеренно не указали ни их стаж, ни доход. Возможно, потому что они решили,что их стаж и доход не является достаточным и это может негативно сказаться на решении банка по одобрению кредита. Или они не являются официально трудоустроенными и не могут подтвердить свой доход. Четкой связи между пропусками и другой информацией найдено не было.

Для заполнения пропусков в столбце `total_income` было выбрано медианное значение. 
Так как медианное значение является более объективным, чем среднее. В силу того, что группы заемщиков достаточно разные, заполнение медианными значениями учитывало образование и тип занятости.

Для столбца `days_employed` было решено пропуски в виде NaN заменить на 0, так как в задании не требуется выявить зависимость между стажем работы и погашение кредита в срок. Стоить заметить, что данные в этом столбце также имеют аномалии и ошибки записи. Замена пропусков в данном случае может сильно исказить результаты. 
Более детальный анализ этого столбца приведен в следующем пункте.

In [8]:
# Приведем столбец 'education' к нижнему регистру
credit_data['education'] = credit_data['education'].str.lower()

In [9]:
# Поиск медианного значения для каждой группы
education_income_pivoted = credit_data.pivot_table(index='income_type',
                                             columns='education', 
                                             values='total_income', 
                                             aggfunc='median')

In [10]:
# Формируем лист для 'education' и 'income_type'
education_list = list(set(credit_data['education']))
income_type_list = list(set(credit_data['income_type']))

In [11]:
# Извлекаем только строки с пропусками
nan_data = credit_data[credit_data['total_income'].isna()]

In [12]:
# Замена пропусков на медианное значение в каждой группе
# Проходим по 'education' и 'income_type' и заполняем пропуски медианой из education_income_pivoted
for education in education_list:
    for income_type in income_type_list:
        if np.isnan(education_income_pivoted.loc[income_type,education]) == False:
            nan_data.loc[(nan_data['education']==education) & 
                     (nan_data['income_type']==income_type),'total_income'] = education_income_pivoted.loc[income_type,education]

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


<div class="alert alert-warning">
<b>Комментарий ревьюера v2⚠️:</b>

Хорошая идея! Но можно было сразу менять в исходном датафрейме, используя `loc` и `isna()` или `fillna()`</div>

In [13]:
# Количество пропусков
credit_data['total_income'].isna().sum()

2174

In [14]:
# Убираем пропуски в исходный данных и проверяем наличие пропусков
credit_data.loc[credit_data['total_income'].isna(),'total_income'] = nan_data['total_income']
credit_data['total_income'].isna().sum()

0

In [15]:
# Заменим пропуски в столбце 'days_employed' на 0
credit_data.loc[credit_data['days_employed'].isna(),'days_employed'] = 0
credit_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b>

<s>1. Неплохое решение заменить пропущенные значения на медиану, но это очень грубое решение. Группы заемщиков достаточно разные, так что лучше заменять на медианы по группам (подумай, какие параметры влияют на доход или стаж).</s>

2. На самом деле, можно подробнее посмотреть на данные стажа. Там четко видно, для каких заемщиков какие проблемы наблюдаются (для кого отрицательные значения, для кого аномально большие). Можно сделать предположения и попытаться исправить эти обе проблемы.</div>

In [16]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

Из обзора данных нам известно, что в колонке `days_employed` есть аномальные значения. А также ошибка записи в данных в виде знака минус. 
В предворительном обзоре данных было замечено, что аномальные значения в данных положительны, а нормальные значения отрицательны. Чтобы проверить это предположение, проверим максимальные и минимальные значения для положительных и отрицательных вхождений столбца `days_employed`. Для удобства `days_employed` приведем к годам (поделим на 365). 

In [17]:
years_employed_positive = credit_data[credit_data['days_employed'] > 0]['days_employed']/365
print('Минимальное значение для положительных вхождений: {0:.5}'.format(years_employed_positive.min()))
print('Максимальное значение для положительных вхождений: {0:.5}'.format(years_employed_positive.max()))

Минимальное значение для положительных вхождений: 900.63
Максимальное значение для положительных вхождений: 1100.7


In [18]:
years_employed_negative = credit_data[credit_data['days_employed'] < 0]['days_employed']/365
print('Минимальное значение для отрицательных вхождений: {0:.5}'.format(years_employed_negative.min()))
print('Максимальное значение для отрицательных вхождений: {0:.5}'.format(years_employed_negative.max()))

Минимальное значение для отрицательных вхождений: -50.381
Максимальное значение для отрицательных вхождений: -0.066141


Предположение подтвердилось: все аномальные значения в столбце `days_employed` являются положительными, а все нормальные значения входят со знаком минус. В целом, можно предположить, что знак минус в столбце `days_employed` является нормой записи или "вшитой" ошибкой (в формуле были перепутаны дата начала и конеца работы, и теперь минус появляется всегда). Наличие аномальных значений может быть следствие неправильного ввода данных/опечаток.

При изучении данных, обнаружилось, что аномальные значения в столбце `days_employed` имеют только два значения `income_type` - безработные и персионеры. Скорее всего данная ошибка связана именно с фактом отсутсвия работы в настоящий момент.


In [19]:
credit_data[credit_data['days_employed'] > 0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Так как природу аномальных значений я не могу предположить, оставим их пока как есть, а отрицательные значения сделаем положительными.

In [20]:
credit_data['days_employed'] = abs(credit_data['days_employed'])
credit_data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Тепрь при необходимиости NaN в столбце `days_employed` может быть заменен на медианное значение.

Колонка `dob_years` имеет тип int, поэтому для проверки быстрой проверки возраста на аномалии можно посмотреть минимальное и максимильное значение в столбце.</b></font>

In [21]:
print('Минимальный возраст заемщика:', credit_data['dob_years'].min())
print('Максимальный возраст заемщика:', credit_data['dob_years'].max())

Минимальный возраст заемщика: 0
Максимальный возраст заемщика: 75


Есть аномали в данных - нулевой возраст. Првоверим, есть ли другие клиенты с возрастом менее 18.

In [22]:
credit_data[credit_data['dob_years']<18]['dob_years'].unique()

array([0])

Связи между аномалиями в возрасте и другими данными обнаружено не было. Скорее всего это ошибки ввода. В задании не требуется исследовать данные на зависимость между возрастом и возвратом кредита в срок. Поэтому аномалию мы не исправляли. Доля аномальных значений в общем объеме данных составляет менее 0.5%, поэтому при необходимости эти строки могут быть исключены из исследования, либо можно попытаться предположить примерный возраст на основе стажа работы и типа занятости клиента.

### Шаг 2.3. Изменение типов данных.

In [23]:
credit_data['total_income'] = credit_data['total_income'].astype(int)

In [24]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Далее необходимо удалить дубликаты. Сначала приведем столбец `education` к единому регистру, это позволит уловить дубликаты, в которых образование написано в разных регистрах.

In [25]:
credit_data['education'] = credit_data['education'].str.lower()
credit_data['education']

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object

Проверка столбцов, неявные дубликаты:

In [26]:
credit_data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [27]:
credit_data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [28]:
credit_data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [29]:
credit_data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [30]:
credit_data['debt'].unique()

array([0, 1])

In [31]:
credit_data['children'].unique()
credit_data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

При проверке остальных столбцов, неявные дубликаты необнаружены.

Зато были обнаружены аномалии в стобце `children` -- `-1` и `20`. Можно предположить что это опечатки и действительное  значения -- `1` и `2`. В силу того, что данное предположение невозмозно проверить, а выборка для клиентов с `1` или `2` детьми достаточно большая (4808 и 2052, соответственно) было принято решение, оставить в  исследовании значения `-1` и `20` без исправлений.

In [32]:
print('Количество явных дубликатов в таблице:')
credit_data.duplicated().sum()

Количество явных дубликатов в таблице:


71

In [33]:
credit_data = credit_data.drop_duplicates().reset_index(drop=True)

In [34]:
print('Количество явных дубликатов в таблице:')
credit_data.duplicated().sum()

Количество явных дубликатов в таблице:


0

**Выводы**

На этапе предобработка обнаружены и справлены следующие проблемы в данных:

- пропущенные значения,
- аномальные значения и ошибки записи
- дубликаты.


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [35]:
# Создание нового датафрейма со столбцами education_id и education
education_dict = credit_data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [36]:
# Создание нового датафрейма со столбцами family_status_id и family_status
family_status_dict = credit_data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [37]:
# Удаление столбцов 'family_status', 'education' из исходного датафрейма
credit_data = credit_data.drop(['family_status', 'education'], axis = 1)
credit_data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

In [38]:
# Функция для присвоения категории на основании дохода.
# Функция использует следующие правила:
#   * 0 - 30000  --  'E';
#   * 30001 - 50000  --  'D';
#   * 50001 - 200000  --  'C';
#   * 200001 - 1000000  --  'B';
#   * 1000001 и выше -- 'A'.

def income_category(income):
    try:
        if income >= 0:
            if   0 <= income <= 30_000: return 'E'
            if  30_001 <= income <= 50_000: return 'D'
            if  50_001 <= income <= 200_000: return 'C'
            if 200_001 <= income <= 1_000_000: return 'B'
            if income > 1_000_001: return 'A'
        else:
            raise ValueError
    except:
        print("Ошибка: Неправильное значение дохода") 

In [39]:
credit_data['total_income_category'] = credit_data['total_income'].apply(income_category)
credit_data.tail()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B
21453,2,1984.507589,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C


### Шаг 2.7. Категоризация целей кредита.

In [40]:
# Функция для присвоения категории на цели кредита.
# Функция делит цели на следующие категории:
#   * ‘операции с автомобилем’,
#   * ‘операции с недвижимостью’,
#   * ‘проведение свадьбы’,
#   * ‘получение образования’.

def purpose_category(purpose):
    try:
        if 'жил' in purpose: return 'операции с недвижимостью'
        elif 'недвиж' in purpose: return 'операции с недвижимостью'
        elif 'автомоб' in purpose: return 'операции с автомобилем'
        elif 'свадь' in purpose: return 'проведение свадьбы'
        elif 'образов' in purpose: return 'получение образования'
        else: return 'цель не определена'
    except:
        print("Ошибка: Неправильное значение цели") 

In [41]:
credit_data['purpose_category'] = credit_data['purpose'].apply(purpose_category)
credit_data.tail()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21450,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем
21453,2,1984.507589,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C,операции с автомобилем


### Ответы на вопросы.

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Столбец `debt` — имел ли задолженность по возврату кредитов. `0` - нет задоженности, `1` - есть задоженность

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?


In [42]:
children_pivot_table = credit_data.pivot_table(index='children', values='debt', aggfunc=('count','sum','mean'))
children_pivot_table.rename( columns={'sum':'number of clients with debt',
                                      'count':'number of clients',
                                      'mean':'% of clients with debt'}, inplace= True)
children_pivot_table = children_pivot_table.drop(index = [-1,20])
children_pivot_table.sort_values('% of clients with debt').style.format({'% of clients with debt': '{:.2%}'})

,number of clients,% of clients with debt,number of clients with debt
children,,,
5,9,0.00%,0
0,14091,7.54%,1063
3,330,8.18%,27
1,4808,9.23%,444
2,2052,9.45%,194
4,41,9.76%,4


##### Вывод 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

Из анализа данных было получено, что клиенты без детей имеют наименьшую долю невозврата кредита в срок. Далее в порядке убывания следуют клиенты с тремя, одним и двумя детьми. 

Однако, различия довольно слабые, к тому же выборка клиентов с 3,4, и 5 детьми очень маленькая. Можно сказать, что зависимость между количесвом детей и возвратом кредита в срок отсутствует или очень слабо выражена

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [43]:
family_pivot_table = credit_data.pivot_table(index='family_status_id',
                                             values='debt', 
                                             aggfunc=('count','sum','mean'))
family_pivot_table.rename( columns={'sum':'number of clients with debt',
                                      'count':'number of clients',
                                      'mean':'% of clients with debt'}, inplace= True)

# добавим в сводную таблицу значение индексов для удобсва
family_pivot_table.insert(loc=0, column='family_status', value=family_status_dict['family_status'])

family_pivot_table.sort_values('% of clients with debt').style.format({'% of clients with debt': '{:.2%}'})

,family_status,number of clients,% of clients with debt,number of clients with debt
family_status_id,,,,
2,вдовец / вдова,959,6.57%,63
3,в разводе,1195,7.11%,85
0,женат / замужем,12339,7.55%,931
1,гражданский брак,4151,9.35%,388
4,Не женат / не замужем,2810,9.75%,274


##### Вывод 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

Наблюдается зависимоть между семейным положением. Можно выделить два кластера. В первый кластер входят клиенты с семейным положением "вдовец/вдова", "в разводе", "женат/замужем" с долей невозврата около 6.5-7.5%. Во второй кластер входят клиенты с семейным положением "гражданский брак", "не женат/не замужем" с долей не возврата около 9.5%

##### Вопрос 3:  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [44]:
income_pivot_table = credit_data.pivot_table(index='total_income_category',
                                             values='debt', 
                                             aggfunc=('count','sum','mean'))
income_pivot_table.rename( columns={'sum':'number of clients with debt',
                                      'count':'number of clients',
                                      'mean':'% of clients with debt'}, inplace= True)

# добавим в сводную таблицу значение индексов для удобсва
income_pivot_table.insert(loc=0, 
                          column='total_income', 
                          value=['1_000_001 и выше', '200_001--1_000_000',
                                 '50_001--200_000','30_001--50_000','0--30_000']
                        )

income_pivot_table.style.format({'% of clients with debt': '{:.2%}'})

,total_income,number of clients,% of clients with debt,number of clients with debt
total_income_category,,,,
A,1_000_001 и выше,25,8.00%,2
B,200_001--1_000_000,5228,6.96%,364
C,50_001--200_000,15829,8.54%,1352
D,30_001--50_000,350,6.00%,21
E,0--30_000,22,9.09%,2


##### Вывод 3:  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Наблюдается нелинейная зависимость между уровнем дохода и возвратом кредита в срок с пиком для дохода категории D (30 001 – 50 000). Однако стоить отметить, что объем выборок достаточный только для категории B и С.

##### Вопрос  4:  Как разные цели кредита влияют на его возврат в срок?


In [45]:
pupose_pivot_table = credit_data.pivot_table(index='purpose_category',
                                             values='debt', 
                                             aggfunc=('count','sum','mean'))
pupose_pivot_table.rename( columns={'sum':'number of clients with debt',
                                      'count':'number of clients',
                                      'mean':'% of clients with debt'}, inplace= True)

pupose_pivot_table.style.format({'% of clients with debt': '{:.2%}'})

,number of clients,% of clients with debt,number of clients with debt
purpose_category,,,
операции с автомобилем,4306,9.36%,403
операции с недвижимостью,10811,7.23%,782
получение образования,4013,9.22%,370
проведение свадьбы,2324,8.00%,186


##### Вывод  4:  Как разные цели кредита влияют на его возврат в срок?

Цели кредита влияют на возврат кредита в срок -- кредиты взятые на операции с недвижимостью и проведение свадьбы имеют наименьшую долю невозврата.

## Общий вывод:

Анализ данных показал, что на факт возврата кредита в срок влияют цели кредита, семейное положение, уровень дохода.

При этом: 
* наибольшую долю возврата имеют кредиты взятые на операции с недвижимостью и проведение свадьбы;
* Клиенты с семейным положением "вдовец/вдова", "в разводе", "женат/замужем" выплачивают кредит в срок чаще, чем клиенты с семейным положением "гражданский брак", "не женат/не замужем";
* зависимость от уровня дохода нелинейная с пиком для дохода категории D (30 001 – 50 000);

А вот зависисмость от количесва детей отсутствует либо очень слабая.



*Важно: в работе не проверялось, являются ли эти отличия статистически значимыми.*


    Так же хочется отметить, что все различия в долях невозврата кредита составляли примерно 1-2%. При полном статистическом анализе, может оказаться, что эти различия не являются значимыми.    
    
В ходе предобработки данных были выявлены ошибки и аномалии: 
   * отрицательный стаж (Предположительно ошибка в программе. Возможно стоит проверить формулу вычисления стажа.)
   * аномально большой стаж для неработающих (Предположительно ошибка в программе.)
   * отрицательное или слишком большое количесво детей, нулевой возраст (Можно добавить проверку на значение при вводе)
   * разный регист в столбце образования (Можно добавить автоматическое приведение к одному регистру)